In [1]:
import torch
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"
import pandas as pd
import numpy as np
from tqdm import tqdm
sys.path.append("../")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from configs.listmle_gsage_nlp import Configs

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [3]:
CFG = Configs()

Valid model types: ['various_splits__nlp_only']
length of train: 405, length of valid: 40, length of test: 34


In [4]:
CFG.load_state_dict(os.path.join(CFG.OUTPUTDIR,"bestmodel_opa.pkl"),map_location="cpu")
model = CFG.model
model.cuda()
model.eval()
1

loading model checkpoint from epoch:  1020


1

In [5]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

total, trainable = count_parameters(model)
print(f"Total parameters: {total}")
print(f"Trainable parameters: {trainable}")


Total parameters: 1262335
Trainable parameters: 1262335


In [6]:
USED_KEYS = ["node_features","node_config_features","node_separation","node_ops","edges","batches"]
pred_sequences = []
for info in tqdm(CFG.test_dataset):
    predictions = []
    for batch in CFG.stream_dataloder_collate([info]):
        with torch.no_grad():
            out = model(**{k:batch[k].cuda() for k in USED_KEYS}).cpu()
            predictions.append(out)
    pred_sequences.append(torch.concat(predictions).flatten()[:len(info["config_runtimes"])])

100%|██████████| 34/34 [12:25<00:00, 21.93s/it]


In [9]:
df = pd.DataFrame(CFG.test_dataset.files,columns=["ID"])
df["ID"] = df.ID.apply(lambda x: "layout:"+x.split("/")[-1].split(".")[0].replace("___",":").replace("test:",""))
df["TopConfigs"] = [";".join([str(x) for x in ps.numpy().argsort().tolist()]) for ps in pred_sequences]

In [10]:
df.to_csv(os.path.join(CFG.OUTPUTDIR,"submission.csv"),index=False)

# Combine multiple files and save

In [11]:
import pandas as pd

In [12]:
files = [
    "/app/nn-runtime-network/workdir/tile_model/results_1697250122338.csv",
    "/app/nn-runtime-network/workdir/listmle_graphsage_xla/submission.csv",
    "/app/nn-runtime-network/workdir/listmle_graphsage_nlp/submission.csv"
]

In [13]:
tdf = pd.concat([pd.read_csv(x) for x in files])
tdf.to_csv(os.path.join("./submission","submission.csv"),index=False)